In [2]:
import os
import pandas as pd
import numpy as np
# def get_file_names(directory):
#     file_names = []
#     folder_names = []
    
#     # 디렉토리의 하위 파일을 탐색합니다.
#     for file in os.listdir(directory):
#         file_path = os.path.join(directory, file)
        
#         # 파일인 경우에만 리스트에 추가합니다.
#         if os.path.isfile(file_path):
#             file_names.append(file)
#         if not os.path.isfile(file_path):
#             folder_names.append(file)
    
#     return file_names, folder_names

def get_file_names(directory):
    file_names = []
    
    # 디렉토리의 하위 파일 및 디렉토리를 탐색합니다.
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            
            # 파일인 경우에만 리스트에 추가합니다.
            if os.path.isfile(file_path):
                file_names.append(file)

    
    return file_names
def reverse_bool(value):
    return not value
# 디렉토리 경로
db_list = ['BS_UPF01_5M', 'GR_UPF06_5M','BS_AMF21_5M', 'BS_AMF22_5M', 'BS_SMF02_5M', 'BS_SMF03_5M',  'DG_UPF01_5M',
           'DJ_AMF21_5M', 'DJ_AMF22_5M', 'DJ_SMF01_5M', 'DJ_SMF03_5M', 'DJ_UPF01_5M', 'DJ_UPF03_5M', 'GJ_UPF01_5M',
           'GR_AMF21_5M', 'GR_AMF22_5M', 'GR_SMF01_5M', 'GR_SMF04_5M', 'GR_UPF01_5M', 'GR_UPF03_5M', 'GR_UPF05_5M',
           'BS_UPF03_5M', 'HH_AMF21_5M', 'HH_AMF22_5M', 'HH_SMF01_5M', 'HH_SMF04_5M', 'HH_UPF01_5M', 'HH_UPF03_5M',
           'HH_UPF04_5M', 'HH_UPF05_5M', 'HH_UPF06_5M']

directory = ['D:/table_df/NF_description/']
bin_file = 0
bin_list = []
entire = 0
entire_index =0
trainable_checker = pd.DataFrame()
for db in db_list[:]:
    db_index_num = 0
    db_num = 0
    # print(db)
    # directory_path_list = [os.path.join(directory[0], db) for i in db_list] #[list1[i] + list2[i] for i in range(len(list1))]
    directory_path=os.path.join(directory[0], db)

    # 파일명 리스트 가져오기
    file_list = get_file_names(directory_path)
    num = 0
    index_num = 0

    # 파일별로 처리
    for file_name in file_list:
        file_path = os.path.join(directory_path, file_name)
        print("file_path :",file_path)
        try:
            tmp_df = pd.read_csv(file_path) #DB_name	table_name	port	col_name	col_min	col_max	col_avg	col_var	shape

        except Exception as e:
            # print("bin file is here",e )
            bin_file += 1
            bin_list.append(file_path)
            continue
            
        try:
            index_num = tmp_df.index[-1] + 1
            
        except IndexError :
            # print("Index Error is happend")
            bin_file += 1
            bin_list.append(file_path)
            continue
        
        # 'shape' 열의 값 파싱하여 비교
        shape_val = tmp_df['shape'].str.replace('(', '').str.replace(',', '').str.replace(')', '').astype(int)
        # print(shape_val < 1000)
        # num += shape_val[shape_val < 1000].shape[0]
    
        # 'col_var' 열의 값 파싱하여 비교
        col_var_val = tmp_df['col_var'].astype(float)#**(1/2)
        col_avg_val = tmp_df['col_avg'].astype(float)
        # print([(shape_val < 1000) | (col_var_val < 0.1)])
        tmp_df['AT_model'] =   ~((shape_val < 1000) | (( col_var_val<0.1) & (col_avg_val < 0.1)))
        num = np.sum((shape_val < 1000) | (( col_var_val<0.1) & (col_avg_val < 0.1)))
        # print("values shape", shape_val)#check
        # print("values var", col_var_val)#check
        # print("values avg", col_avg_val)#check
        # print("values div:",col_var_val/ col_avg_val)#check
        # print(np.sum((shape_val < 1000)))#check
        # print(np.sum(( col_var_val/ col_avg_val < 0.1)))#check
        # print( tmp_df.head()) #check
        trainable_checker = pd.concat([trainable_checker,tmp_df],axis=0)
        
        
        
        # 결과 출력 포트 합산
        if file_name in ["PAG_SM1.csv","PAG_SM2.csv","REG_IPLMN.csv"]:
            index_num =110
            num = 0
        
        # print("총 통계지표 개수:", index_num)
        # print("비학습 통계지표 개수:", num)
        # print("비학습 률:", num/index_num *100)

        db_index_num += index_num+0.000001
        db_num += num
        
        entire_index += index_num
        entire += num
    ###########
    ## 
    if db in 'HH_UPF04_5M':
        db_index_num += 0.0000001

    print("DB 이름                :",db)
    print("DB 통계지표 개수       :", db_index_num)
    print("DB 학습 통계지표 개수  :", db_index_num-db_num)
    print("DB 비학습 통계지표 개수:", db_num)
    print("Null 테이블 개수       :", bin_file)
    # print("DB 비학습률            :", db_num/db_index_num *100)
    print("-------------------------------------------")
    # break
print("-------------------------------------------")    

print("DB 전체 총 통계지표 개수    :", entire_index)
print("DB 전체 학습 통계지표 개수    :", entire_index-entire)
print("DB 전체 비학습 통계지표 개수:", entire)
# print("DB 전체 비학습 률           :", entire/entire_index *100)
trainable_checker.to_csv(os.path.join(directory[0], "trainable_checker.csv") )
        


DB 이름                : BS_UPF01_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : GR_UPF06_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : BS_AMF21_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : BS_AMF22_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : BS_SMF02_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : BS_SMF03_5M
DB 통계지표 개수       : 0
DB 학습 통계지표 개수  : 0
DB 비학습 통계지표 개수: 0
Null 테이블 개수       : 0
-------------------------------------------
DB 이름                : DG_UPF01_5M
DB 통계지표 개수 

OSError: Cannot save file into a non-existent directory: 'D:/table_df/NF_description'

In [3]:
jk

NameError: name 'jk' is not defined